In [1]:
import pandas as pd
import json

df_scores = pd.read_csv("scores.csv")
df_activities = pd.read_excel("activities.xlsx", sheet_name="HoatDong")
with open("profiles.json", "r", encoding="utf-8") as f:
    profiles = json.load(f)
df_profiles = pd.json_normalize(profiles)

FileNotFoundError: [Errno 2] No such file or directory: 'scores.csv'

In [ ]:
df_scores.info()
df_scores.describe()
df_scores.isna().sum()

In [ ]:
df_scores['GioiTinh'] = df_scores['GioiTinh'].str.strip().str.lower().map({
    'nam':'Nam','male':'Nam','m':'Nam',
    'nữ':'Nữ','nu':'Nữ','female':'Nữ','f':'Nữ'
})

In [ ]:
df_scores['DiemCuoiKy'] = df_scores.groupby('Lop')['DiemCuoiKy'].transform(lambda x: x.fillna(x.median()))

In [ ]:
df_profiles['NgaySinh'] = pd.to_datetime(df_profiles['NgaySinh'], errors='coerce')
df_profiles['NamSinh'] = df_profiles['NgaySinh'].dt.year

In [ ]:
Q1 = df_scores['DiemCuoiKy'].quantile(0.25)
Q3 = df_scores['DiemCuoiKy'].quantile(0.75)
IQR = Q3 - Q1
df_scores = df_scores[(df_scores['DiemCuoiKy'] >= Q1 - 1.5*IQR) &
(df_scores['DiemCuoiKy'] <= Q3 + 1.5*IQR)]

In [ ]:
df_final = df_scores.merge(df_activities, on='MaSV', how='left')
df_final = df_final.merge(df_profiles, on='MaSV', how='left')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Trung bình điểm theo khoa
print(df_final.groupby("Khoa")["DiemCuoiKy"].mean())

In [ ]:
# So sánh phân bố
sns.kdeplot(df_final['DiemGiuaKy'], label="Giữa kỳ")
sns.kdeplot(df_final['DiemCuoiKy'], label="Cuối kỳ")
plt.legend()
plt.show()

In [ ]:
# Heatmap tương quan
sns.heatmap(df_final[['DiemGiuaKy','DiemCuoiKy','DiemChuyenCan']].corr(),
annot=True, cmap="coolwarm")
plt.show()

In [ ]:
df_final.to_csv("students_final.csv", index=False)